In [494]:
#Import Depedencies
import pandas as pd
import csv

In [495]:
csvPath = "../Resources/video_game.csv"

In [496]:
#Read video_game.csv and store in Pandas DataFrame
game_df = pd.read_csv(csvPath)
game_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [523]:
#Count number of unique players
totalPlayers_dict = {}

totalPlayers = game_df["SN"].nunique()
totalPlayers_dict = {"Total Players": [totalPlayers]}

pd.DataFrame(totalPlayers_dict)

,Total Players
0,576


In [498]:
#Purchasing Analysis

uniqueItems = game_df["Item Name"].nunique()
print(f"Number of unique items: {uniqueItems}")
avgPrice = round(game_df["Price"].mean(),2)
print(f"Average purchase price: ${avgPrice}")
numItems = game_df["Item Name"].count()
print(f"Number of Purchases: {numItems}")
revenue = game_df["Price"].sum()
print(f"Total revenue: ${revenue}")

pd.DataFrame([{"Number of Unique Items": uniqueItems,
              "Average Price": avgPrice, 
               "Number of Purchases": numItems,
               "Total Revenue": revenue,}])

Number of unique items: 179
Average purchase price: $3.05
Number of Purchases: 780
Total revenue: $2379.77


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.05,780,2379.77


In [526]:
#Read rows from csv into lists used in later analysis
unique_users = []
genders =[]
ages = []
unique_items_list = []
unique_items_dict = {}
unique_items = []

with open(csvPath) as csvFile:
    csvReader = csv.reader(csvFile,delimiter=",")
    header = next(csvReader,None)
    for row in csvReader:
        SN = row[1]
        gender = row[3]
        age = row[2]
        itemID = int(row[4])
        itemName = row[5]
        itemPrice = row[6]
        if SN not in unique_users:
            unique_users.append(SN)
            genders.append(gender)
            ages.append(age)
        if itemID not in unique_items_list:
            unique_items_list.append(itemID)
            unique_items_dict = {"Item ID": itemID, "Item Name": itemName, "Item Price":itemPrice}
            unique_items.append(unique_items_dict)

In [535]:
#Create unique_users_df (holds single row for each player's SN, Age)
unique_users_dict = {}
unique_users_list=[]

for i in range(len(unique_users)):
    SN = unique_users[i]
    age = int(ages[i])
    unique_users_dict = {"SN":SN , "Age":age}
    unique_users_list.append(unique_users_dict)
    
unique_user_df = pd.DataFrame(unique_users_list)
unique_user_df

,SN,Age
0,Lisim78,20
1,Lisovynya38,40
2,Ithergue48,24
3,Chamassasya86,24
4,Iskosia90,23
...,...,...
571,Hala31,21
572,Jiskjask80,11
573,Aethedru70,21
574,Yathecal72,20


In [538]:
#Create unique_items_df (holds single row for each item's ItemID, Item Name, Price)

unique_items_df = pd.DataFrame(unique_items)
unique_items_df["Item ID"]=unique_items_df["Item ID"].astype(int)
unique_items_df.set_index("Item ID")

,Item Name,Item Price
Item ID,,
108,"Extraction, Quickblade Of Trembling Hands",3.53
143,Frenzied Scimitar,1.56
92,Final Critic,4.88
100,Blindscythe,3.27
131,Fury,1.44
...,...,...
47,"Alpha, Reach of Ending Hope",3.58
130,Alpha,2.07
90,Betrayer,2.94


In [499]:
#Gender Demographics

maleCount = genders.count("Male")
malePerc = round(maleCount/totalPlayers*100,2)
femaleCount = genders.count("Female")
femalePerc = round(femaleCount/totalPlayers*100,2)
otherCount = genders.count("Other / Non-Disclosed")
otherPerc = round(otherCount/totalPlayers*100,2)
            
gender_dicts = [{"Total Count": maleCount, "Gender": "Male", "Percentage of Players":malePerc},
               {"Total Count": femaleCount, "Gender": "Female", "Percentage of Players":femalePerc},
               {"Total Count": otherCount, "Gender": "Other / Non-Disclosed", "Percentage of Players":otherPerc}]


gender_df = pd.DataFrame(gender_dicts)
gender_df = gender_df.set_index("Gender")
gender_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [501]:
#Purchasing Analystics (Gender)
malePurchases_df = game_df.loc[game_df["Gender"]=="Male",:]
malePurchases = malePurchases_df["Item Name"].count()
avgMalePrice = round(malePurchases_df["Price"].mean(),2)
totalMalePrice = round(malePurchases_df["Price"].sum(),2)
avgPricePerMale = round(totalMalePrice/maleCount,2)

femalePurchases_df = game_df.loc[game_df["Gender"]=="Female",:]
femalePurchases = femalePurchases_df["Item Name"].count()
avgFemalePrice = round(femalePurchases_df["Price"].mean(),2)
totalFemalePrice = round(femalePurchases_df["Price"].sum(),2)
avgPricePerFemale = round(totalFemalePrice/femaleCount,2)

otherPurchases_df = game_df.loc[game_df["Gender"]=="Other / Non-Disclosed",:]
otherPurchases = otherPurchases_df["Item Name"].count()
avgOtherPrice = round(otherPurchases_df["Price"].mean(),2)
totalOtherPrice = round(otherPurchases_df["Price"].sum(),2)
avgPricePerOther = round(totalOtherPrice/otherCount,2)

gender_purchase_dicts = [{"Purchase Count": malePurchases, "Gender": "Male", "Average Purchase Price":avgMalePrice, "Total Purchase Value":totalMalePrice, "Avg Total Purchase per Person":avgPricePerMale},
                        {"Purchase Count": femalePurchases, "Gender": "Female", "Average Purchase Price":avgFemalePrice, "Total Purchase Value":totalFemalePrice, "Avg Total Purchase per Person":avgPricePerFemale},
                        {"Purchase Count": otherPurchases, "Gender": "Other / Non-Disclosed", "Average Purchase Price":avgOtherPrice, "Total Purchase Value":totalOtherPrice, "Avg Total Purchase per Person":avgPricePerOther}]
gender_purchase_df = pd.DataFrame(gender_purchase_dicts)
gender_purchase_df = gender_purchase_df.set_index("Gender")
gender_purchase_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Male,652,3.02,1967.64,4.07
Female,113,3.20,361.94,4.47
Other / Non-Disclosed,15,3.35,50.19,4.56


In [536]:
#Age Demographics
#Bin Age Ranges to unique_user_df (want to know age range for each unique player)
percent_list = []
age_count = []

bins = [0,9.9,14.9,19.9,24.9,29.9,34.9,39.9,50]
age_ranges = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

unique_user_df["Age Ranges"] = pd.cut(unique_user_df["Age"], bins, labels=age_ranges)
unique_user_df = unique_user_df.drop("Age",1)


playerPerc = round(1/576*100,2)
for i in range(576):
    percent_list.append(playerPerc)
    age_count.append(1)

unique_user_df["Percentage of Players"] = percent_list
unique_user_df["Player Count"] = age_count
#unique_user_df.reset_index()

grouped_age_df = unique_user_df.groupby("Age Ranges")
grouped_age_df.sum()

,Percentage of Players,Player Count
Age Ranges,,
<10,2.89,17
10-14,3.74,22
15-19,18.19,107
20-24,43.86,258
25-29,13.09,77
30-34,8.84,52
35-39,5.27,31
40+,2.04,12


In [504]:
#Bin Age Ranges for all rows in game_df for Purchasing Analysis (Age)
bins = [0,9.9,14.9,19.9,24.9,29.9,34.9,39.9,50]
age_ranges = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

game_df["Age Ranges"] = pd.cut(game_df["Age"], bins, labels=age_ranges)
game_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Ranges
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24
...,...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54,20-24
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63,20-24
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46,20-24
778,778,Sisur91,7,Male,92,Final Critic,4.19,<10


In [506]:
#Purchasing Analysis (Age)
age_ranges = []

for item in unique_user_df["Age Ranges"]:
    age_ranges.append(item)

lessThan10Purchases_df = game_df.loc[game_df["Age Ranges"]=="<10",:]
lessThan10Purchases = lessThan10Purchases_df["Item Name"].count()
avgLessThan10Price = round(lessThan10Purchases_df["Price"].mean(),2)
totalLessThan10Price = round(lessThan10Purchases_df["Price"].sum(),2)
lessThan10Count = age_ranges.count("<10")
avgPricePerLessThan10 = round(totalLessThan10Price/lessThan10Count,2)


tenFourteenPurchases_df = game_df.loc[game_df["Age Ranges"]=="10-14",:]
tenFourteenPurchases = tenFourteenPurchases_df["Item Name"].count()
avgTenFourteenPrice = round(tenFourteenPurchases_df["Price"].mean(),2)
totalTenFourteenPrice = round(tenFourteenPurchases_df["Price"].sum(),2)
tenFourteenCount = age_ranges.count("10-14")
avgPricePerTenFourteen = round(totalTenFourteenPrice/tenFourteenCount,2)


fifteenNineteenPurchases_df = game_df.loc[game_df["Age Ranges"]=="15-19",:]
fifteenNineteenPurchases = fifteenNineteenPurchases_df["Item Name"].count()
avgFifteenNineteenPrice = round(fifteenNineteenPurchases_df["Price"].mean(),2)
totalFifteenNineteenPrice = round(fifteenNineteenPurchases_df["Price"].sum(),2)
fifteenNineteenCount = age_ranges.count("15-19")
avgPricePerFifteenNineteen = round(totalFifteenNineteenPrice/fifteenNineteenCount,2)


twenty24Purchases_df = game_df.loc[game_df["Age Ranges"]=="20-24",:]
twenty24Purchases = twenty24Purchases_df["Item Name"].count()
avgTwenty24Price = round(twenty24Purchases_df["Price"].mean(),2)
totalTwenty24Price = round(twenty24Purchases_df["Price"].sum(),2)
twenty24Count = age_ranges.count("20-24")
avgPricePerTwenty24 = round(totalTwenty24Price/twenty24Count,2)


twentyfive29Purchases_df = game_df.loc[game_df["Age Ranges"]=="25-29",:]
twentyfive29Purchases = twentyfive29Purchases_df["Item Name"].count()
avgTwentyfive29Price = round(twentyfive29Purchases_df["Price"].mean(),2)
totalTwentyfive29Price = round(twentyfive29Purchases_df["Price"].sum(),2)
twentyfive29Count = age_ranges.count("25-29")
avgPricePerTwentyfive29 = round(totalTwentyfive29Price/twentyfive29Count,2)


thirty34Purchases_df = game_df.loc[game_df["Age Ranges"]=="30-34",:]
thirty34Purchases = thirty34Purchases_df["Item Name"].count()
avgThirty34Price = round(thirty34Purchases_df["Price"].mean(),2)
totalThirty34Price = round(thirty34Purchases_df["Price"].sum(),2)
thirty34Count = age_ranges.count("30-34")
avgPricePerThirty34 = round(totalThirty34Price/thirty34Count,2)


thirtyfive39Purchases_df = game_df.loc[game_df["Age Ranges"]=="35-39",:]
thirtyfive39Purchases = thirtyfive39Purchases_df["Item Name"].count()
avgThirtyfive39Price = round(thirtyfive39Purchases_df["Price"].mean(),2)
totalThirtyfive39Price = round(thirtyfive39Purchases_df["Price"].sum(),2)
thirtyfive39Count = age_ranges.count("35-39")
avgPricePerThirtyfive39 = round(totalThirtyfive39Price/thirtyfive39Count,2)


fortyPurchases_df = game_df.loc[game_df["Age Ranges"]=="40+",:]
fortyPurchases = fortyPurchases_df["Item Name"].count()
avgFortyPrice = round(fortyPurchases_df["Price"].mean(),2)
totalFortyPrice = round(fortyPurchases_df["Price"].sum(),2)
fortyCount = age_ranges.count("40+")
avgPricePerForty = round(totalFortyPrice/fortyCount,2)


age_purchase_dicts = [{"Purchase Count": lessThan10Purchases, "Age Ranges": "<10", "Average Purchase Price":avgLessThan10Price, "Total Purchase Value":totalLessThan10Price, "Avg Total Purchase per Person":avgPricePerLessThan10},
                     {"Purchase Count": tenFourteenPurchases, "Age Ranges": "10-14", "Average Purchase Price":avgTenFourteenPrice, "Total Purchase Value":totalTenFourteenPrice, "Avg Total Purchase per Person":avgPricePerTenFourteen},
                     {"Purchase Count": fifteenNineteenPurchases, "Age Ranges": "15-19", "Average Purchase Price":avgFifteenNineteenPrice, "Total Purchase Value":totalFifteenNineteenPrice, "Avg Total Purchase per Person":avgPricePerFifteenNineteen},
                     {"Purchase Count": twenty24Purchases, "Age Ranges": "20-24", "Average Purchase Price":avgTwenty24Price, "Total Purchase Value":totalTwenty24Price, "Avg Total Purchase per Person":avgPricePerTwenty24},
                     {"Purchase Count": twentyfive29Purchases, "Age Ranges": "25-29", "Average Purchase Price":avgTwentyfive29Price, "Total Purchase Value":totalTwentyfive29Price, "Avg Total Purchase per Person":avgPricePerTwentyfive29},
                     {"Purchase Count": thirty34Purchases, "Age Ranges": "30-34", "Average Purchase Price":avgThirty34Price, "Total Purchase Value":totalThirty34Price, "Avg Total Purchase per Person":avgPricePerThirty34},
                     {"Purchase Count": thirtyfive39Purchases, "Age Ranges": "35-39", "Average Purchase Price":avgThirtyfive39Price, "Total Purchase Value":totalThirtyfive39Price, "Avg Total Purchase per Person":avgPricePerThirtyfive39},
                     {"Purchase Count": fortyPurchases, "Age Ranges": "40+", "Average Purchase Price":avgFortyPrice, "Total Purchase Value":totalFortyPrice, "Avg Total Purchase per Person":avgPricePerForty}]

age_purchase_df = pd.DataFrame(age_purchase_dicts)
age_purchase_df = age_purchase_df.set_index("Age Ranges")
age_purchase_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,3.35,77.13,4.54
10-14,28,2.96,82.78,3.76
15-19,136,3.04,412.89,3.86
20-24,365,3.05,1114.06,4.32
25-29,101,2.90,293.00,3.81
30-34,73,2.93,214.00,4.12
35-39,41,3.60,147.67,4.76
40+,13,2.94,38.24,3.19


In [507]:
#Top Spenders
spenders_list = []
spenders_dict = {}

for i in range(len(unique_users)):
    SN = unique_users[i]
    individual_player_df = game_df.loc[game_df["SN"]==SN,:]
    purchases = individual_player_df["Item ID"].count()
    totalPurchasePrice = individual_player_df["Price"].sum()
    avgPurchases = round(totalPurchasePrice/purchases,2)
    spenders_dict = {"SN":SN, "Purchase Count":purchases, "Average Purchase Price":avgPurchases, "Total Purchase Value":totalPurchasePrice}
    spenders_list.append(spenders_dict)

spenders_df = pd.DataFrame(spenders_list)
spenders_df = spenders_df.sort_values("Total Purchase Value",ascending = False)

spenders_df = spenders_df.set_index("SN")
spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


In [517]:
#Most Popular Items

items_list = []
items_dict = {}
item_name_list = []

for i in range(len(unique_items_list)):
    itemID = int(unique_items_list[i])
    individual_item_df = game_df.loc[game_df["Item ID"]==itemID,:]
    itemPurchases = individual_item_df["Item Name"].count()
    totalItemPurchasePrice = individual_item_df["Price"].sum()
    items_dict = {"Item ID":itemID, "Purchase Count":itemPurchases, "Total Purchase Value":totalItemPurchasePrice}
    items_list.append(items_dict)

items_df = pd.DataFrame(items_list)
items_df = items_df.set_index("Item ID")

top_items_merged_df = pd.merge(unique_items_df,items_df,on="Item ID")
top_items_merged_df = top_items_merged_df.set_index("Item ID")
top_items_merged_df = top_items_merged_df.sort_values(["Purchase Count"],ascending = False)
top_items_merged_df.head(6)

,Item Name,Item Price,Purchase Count,Total Purchase Value
Item ID,,,,
92,Final Critic,4.88,13,59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",4.23,12,50.76
108,"Extraction, Quickblade Of Trembling Hands",3.53,9,31.77
82,Nirvana,4.9,9,44.10
145,Fiery Glass Crusader,4.58,9,41.22
132,Persuasion,3.19,9,28.99


In [520]:
#Most Profitable Items

most_profitable_df = top_items_merged_df.sort_values("Total Purchase Value",ascending=False)
most_profitable_df.head()

,Item Name,Item Price,Purchase Count,Total Purchase Value
Item ID,,,,
92,Final Critic,4.88,13,59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",4.23,12,50.76
82,Nirvana,4.9,9,44.10
145,Fiery Glass Crusader,4.58,9,41.22
103,Singed Scalpel,4.35,8,34.80
